# Pandas tip #15: Compress your data!
When working with data you often come accross CSV files. The great thing about CSV files is that they are human readable. The bad thing is that they are not very space efficient. My experience is that the 'human readable' benefit is only a marginal benefit when working with that file for the first time.

A nice CSV feature of Pandas is to store your data in a compressed way using the `compression` parameter. Out of the box Pandas can us zip/gzip, bz2, and xz compression when storing a DataFrame. As we are generally using the '.csv.gz' as an extention, Pandas can automatically infer the compression, therefore, we can omit the `compression` parameter.

As we are compressing the data, it takes a bit more effort to store the data. Obviously, the compressed file is not human readable anymore. Under the hood, Pandas uses the gzip, bzip2, and the xz library and streams the text lines to the compressed file. The most efficient is bzip2, which reduces the size to one third of the original size while taking three times longer.

Writing the data line by line is of course not very efficient and as we loose the "benefit" of human readability anyhow, we could also store it as Parquet. Parquet is a column format and highly optimized for reading, i.e. writing takes a bit more effort. Still, compared to a method that stores line by line, Parquet is blazingly fast and until now, I have not yet seen any real downsides of using it.

Parquet is by far my favorite format and I highly recommend it!

Lets generate some random data:

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path

rng = np.random.default_rng(42)
n_rows, n_cols = 100_000, 100

df = pd.DataFrame(
    rng.integers(0,1000, size=(n_rows, n_cols)),
    columns=[str(x) for x in np.arange(n_cols)]
)

Lets save this to a regular csv:

In [ ]:
def get_filesize(f: Path) -> str:
    file_size = f.stat().st_size
    for unit in ['','K','M','G','T']:
        if file_size < 1024:
            return f"{file_size:3.1f}{unit}B"
        file_size /= 1024

In [ ]:
%%time
large_file = Path('large_file.csv')
df.to_csv(large_file)
print(f'CSV file size: {get_filesize(large_file)}')

Using gzip compression:

In [ ]:
gzip_file = Path('gzipped_file.csv.gz')

In [ ]:
%%time
df.to_csv(gzip_file, compression='gzip')

In [ ]:
print(f'GZ CSV file size: {get_filesize(gzip_file)}')

The default option for `compression` is 'infer' which detects which type of compression is used from the extention. Therefore, we only need to supply the .gz extentions and it will automatically gzip the file.

In [ ]:
bzip_file = Path('bzipped_file.csv.bz2')

In [ ]:
%%time
df.to_csv(bzip_file)

In [ ]:
print(f'BZ2 CSV file size: {get_filesize(bzip_file)}')

In [ ]:
xzip_file = Path('xzipped_file.csv.xz')

In [ ]:
%%time
df.to_csv(xzip_file)

In [ ]:
print(f'XZ CSV file size: {get_filesize(xzip_file)}')

In [ ]:
parquet_file = Path('parquet_file.parquet')

In [ ]:
%%time
df.to_parquet(parquet_file)

In [ ]:
print(f'Parquet file size: {get_filesize(parquet_file)}')

If you have any questions, comments, or requests, feel free to [contact me on LinkedIn](https://linkedin.com/in/dennisbakhuis).